<a href="https://colab.research.google.com/github/BDH-teacher/Deep_Learning_Audit_code/blob/main/InstructGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install -U transformers accelerate sentencepiece
!pip -q install -U openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.6 MB/s eta 0:00:00


In [2]:
import os
import torch

print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

torch: 2.9.0+cu126
cuda available: True
device: cuda


In [3]:
from transformers import pipeline

# HF docs: pipeline("...") 패턴 그대로 사용 :contentReference[oaicite:2]{index=2}
pipe = pipeline(
    "text2text-generation",            # HF docs에 명시된 task 중 하나 :contentReference[oaicite:3]{index=3}
    model="google/flan-t5-small",
    device=0 if torch.cuda.is_available() else -1
)

examples = [
    "Translate English to French: I love machine learning.",
    "Summarize: Transformers are neural networks that use attention mechanisms to model dependencies in sequences.",
    "Answer the question: What is RLHF in one sentence?",
]

for x in examples:
    out = pipe(x, max_new_tokens=64, do_sample=False)
    print(f"\n[INSTRUCTION]\n{x}\n[OUTPUT]\n{out[0]['generated_text']}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0



[INSTRUCTION]
Translate English to French: I love machine learning.
[OUTPUT]
J'ai envie de l'apprentissage des machines.

[INSTRUCTION]
Summarize: Transformers are neural networks that use attention mechanisms to model dependencies in sequences.
[OUTPUT]
Transformers are a type of neural network that uses attention mechanisms to model dependencies in sequences.

[INSTRUCTION]
Answer the question: What is RLHF in one sentence?
[OUTPUT]
RLHF is a unit of measurement of the size of a unit of measurement of the size of a unit of measurement of the size of a unit of measurement of the size of a unit of measurement of the size of a unit of measurement of the size of a unit of measurement of


In [4]:
import os
from openai import OpenAI

# (key는 환경변수) :contentReference[oaicite:7]{index=7}
# os.environ["OPENAI_API_KEY"] = "YOUR_KEY"

client = OpenAI()
GPT_MODEL = "gpt-4o-mini"  # 예시 모델명 :contentReference[oaicite:8]{index=8}

if not os.getenv("OPENAI_API_KEY"):
    print("OPENAI_API_KEY가 없어 API 호출은 스킵합니다.")
else:
    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[{"role": "user", "content": "Hello!"}],
        temperature=0,
    )
    print(response.choices[0].message.content)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [5]:
def get_completion(prompt: str, model: str = "gpt-4o-mini"):
    if not os.getenv("OPENAI_API_KEY"):
        print("OPENAI_API_KEY가 없어 get_completion 호출을 스킵합니다.")
        return None

    from openai import OpenAI
    client = OpenAI()

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    return response.choices[0].message.content

print(get_completion("Say 'this is a test'."))  # 키 없으면 스킵

OPENAI_API_KEY가 없어 get_completion 호출을 스킵합니다.
None


In [6]:
def chat_with_system(system_prompt: str, user_prompt: str, model: str = "gpt-4o-mini"):
    if not os.getenv("OPENAI_API_KEY"):
        print("OPENAI_API_KEY가 없어 chat 호출을 스킵합니다.")
        return None

    from openai import OpenAI
    client = OpenAI()

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},  # 메시지 구조 :contentReference[oaicite:11]{index=11}
            {"role": "user", "content": user_prompt},
        ],
        temperature=0,
    )
    return response.choices[0].message.content

system_prompt = "You are a helpful assistant."
user_prompt = "Explain InstructGPT in 2 bullets."
print(chat_with_system(system_prompt, user_prompt))


OPENAI_API_KEY가 없어 chat 호출을 스킵합니다.
None


In [7]:
tweet = "I loved the new movie. It was amazing!"

system_prompt = (
    "Your task is to answer whether a tweet is positive or negative. "
    "Output only POSITIVE or NEGATIVE."
)

print(chat_with_system(system_prompt, tweet))

OPENAI_API_KEY가 없어 chat 호출을 스킵합니다.
None
